In [ ]:
"""
### Overview:
This script is designed for training an image classification model using a fine-tuned ResNet-50, incorporating data preprocessing,
augmentation, and hyperparameter optimization via Bayesian methods.

### Key Steps:

1. Label Encoding:
   - Converts categorical labels from the dataset into numerical values.
   - Saves the mapping dictionary (`label_to_idx.json`) to ensure consistency across datasets.

2. Custom Dataset Class (`AppleDataset`):
   - Loads image paths and their corresponding labels from CSV files.
   - Facilitates easy integration with PyTorch’s `DataLoader`.

3. Data Augmentation:
   - Applies random transformations (cropping, flipping, rotation, color jittering) to the training images.
   - Aims to enhance model robustness and reduce overfitting.

4. Model Setup (ResNet-50 Fine-tuning):
   - Uses a pre-trained ResNet-50 as the feature extractor.
   - Unfreezes the last few layers to allow fine-tuning.
   - Replaces the final classification layer with a custom fully connected layer, including dropout for regularization.

5. Bayesian Optimization (Optuna for Hyperparameter Tuning):
   - Searches for the best hyperparameters to maximize validation accuracy.
   - Tunable parameters include:
     - **Batch size:** {16, 32, 64}
     - **Learning rate:** Continuous range from 1e-5 to 1e-3
     - **Dropout rate:** Continuous range from 0.2 to 0.7
     - **Weight decay:** Continuous range from 1e-5 to 1e-3

6. Training Process:
   - Each trial runs for a maximum of **10 epochs**.
   - Implements **early stopping** to halt training if validation accuracy plateaus.

7. Model Selection & Saving:
   - The best-performing model from all trials is saved as `best_resnet50_model.pth`.
   - Ensures optimal performance on unseen data.

This pipeline efficiently optimizes and fine-tunes the ResNet-50 model while leveraging Bayesian optimization for better hyperparameter selection.
"""

!pip install torch torchvision pandas pillow scikit-learn optuna tqdm
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import pandas as pd
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
import optuna
from optuna.samplers import TPESampler
from tqdm import tqdm
import requests
from io import BytesIO
import torch.cuda.amp as amp  # Import for mixed precision training

from google.colab import drive
drive.mount('/content/gdrive')

# Define paths
absolute_path = "/content/gdrive/My Drive/Projects/Multimodal/"
TRAIN_CSV = absolute_path + "Datasets/Apple_train_set_100.csv"
VAL_CSV = absolute_path + "Datasets/Apple_validation_set_100.csv"
LABEL_JSON = absolute_path + "Datasets/label_to_idx-100.json"
BEST_MODEL_PATH = absolute_path + "Datasets/best_resnet50_model-100.pth"

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Print GPU info
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory Allocated: {torch.cuda.memory_allocated(0)/1e9:.2f} GB")
    print(f"Memory Reserved: {torch.cuda.memory_reserved(0)/1e9:.2f} GB")
    print(f"Total Memory: {torch.cuda.get_device_properties(0).total_memory/1e9:.2f} GB")

BATCH_SIZES = [16, 32, 64, 128, 256]
NUM_WORKERS = 8
PREFETCH_FACTOR = 2

# Set up data prefetching and pinning
torch.multiprocessing.set_sharing_strategy('file_system')
torch.set_float32_matmul_precision('high')  # Use TF32 precision on A100

# Prepare data downloading and caching
class ImageCache:
    def __init__(self, capacity=1000):
        self.capacity = capacity
        self.cache = {}

    def get(self, url):
        if url in self.cache:
            return self.cache[url]

        try:
            response = requests.get(url)
            image = Image.open(BytesIO(response.content)).convert('RGB')

            # Keep cache size in check
            if len(self.cache) >= self.capacity:
                # Remove a random item
                self.cache.pop(next(iter(self.cache)))

            self.cache[url] = image
            return image
        except Exception as e:
            print(f"Error loading image from {url}: {e}")
            return Image.new('RGB', (224, 224), color='black')

# Global image cache
image_cache = ImageCache()

# Custom Dataset with optimized loading
class AppleDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

        # Load the label encoding
        with open(LABEL_JSON, 'r') as f:
            self.label_to_idx = json.load(f)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_url = self.data.iloc[idx]['Image']
        category = self.data.iloc[idx]['Category']

        # Convert category to encoded label
        label = self.label_to_idx[category]

        # Load image from URL with caching
        image = image_cache.get(img_url)

        if self.transform:
            image = self.transform(image)

        return image, label

# Function to create the dataset and dataloaders
def create_data_loaders(batch_size):
    # Define transformations
    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    val_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Create datasets
    train_dataset = AppleDataset(TRAIN_CSV, transform=train_transform)
    val_dataset = AppleDataset(VAL_CSV, transform=val_transform)

    # Create dataloaders with optimized settings
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True,
        prefetch_factor=PREFETCH_FACTOR,
        persistent_workers=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True,
        prefetch_factor=PREFETCH_FACTOR,
        persistent_workers=True
    )

    return train_loader, val_loader

# Function to perform label encoding
def perform_label_encoding():
    train_data = pd.read_csv(TRAIN_CSV)
    val_data = pd.read_csv(VAL_CSV)

    # Combine all categories
    all_categories = pd.concat([train_data['Category'], val_data['Category']]).unique()

    # Create encoding
    label_encoder = LabelEncoder()
    label_encoder.fit(all_categories)

    # Create label_to_idx dictionary
    label_to_idx = {category: int(idx) for category, idx in zip(all_categories, label_encoder.transform(all_categories))}

    # Save to json
    with open(LABEL_JSON, 'w') as f:
        json.dump(label_to_idx, f)

    print(f"Label encoding saved to {LABEL_JSON}")
    return len(label_to_idx)

# Function to train for one epoch with mixed precision
def train_epoch(model, loader, criterion, optimizer, device, scaler):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in tqdm(loader, desc="Training"):
        inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)

        # Zero the parameter gradients
        optimizer.zero_grad(set_to_none=True)  # More efficient than zero_grad()

        # Forward pass with mixed precision
        with amp.autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        # Backward and optimize with scaled gradients
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # Statistics
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# Function to validate with mixed precision
def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad(), amp.autocast():
        for inputs, labels in tqdm(loader, desc="Validating"):
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Statistics
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = running_loss / total
    val_acc = correct / total
    return val_loss, val_acc

# Function to initialize the model
def create_model(num_classes, dropout_rate=0.5):
    # Load with higher performance settings
    try:
        model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
    except:
        model = models.resnet50(pretrained=True)

    # Freeze early layers
    for param in list(model.parameters())[:-4]:  # Freeze fewer layers
        param.requires_grad = False

    # Replace the final fully connected layer
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(dropout_rate),
        nn.Linear(num_ftrs, num_classes)
    )

    # Use channels_last memory format for better performance on A100
    model = model.to(device, memory_format=torch.channels_last)
    return model

# Objective function for Optuna
def objective(trial, num_classes):
    # Define hyperparameters to optimize
    batch_size = trial.suggest_categorical('batch_size', BATCH_SIZES)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.7)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True)

    # Create model and dataloaders
    model = create_model(num_classes, dropout_rate)
    train_loader, val_loader = create_data_loaders(batch_size)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # Create gradient scaler for mixed precision training
    scaler = amp.GradScaler()

    # Train for a few epochs
    best_val_acc = 0
    patience = 0
    max_patience = 3

    for epoch in range(10):  # Maximum 10 epochs per trial
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device, scaler)
        val_loss, val_acc = validate(model, val_loader, criterion, device)

        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

        # Print GPU memory usage
        if torch.cuda.is_available():
            print(f"GPU Memory: {torch.cuda.memory_allocated(0)/1e9:.2f}GB / {torch.cuda.get_device_properties(0).total_memory/1e9:.2f}GB")

        trial.report(val_acc, epoch)

        # Handle pruning
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience = 0

            # Save the current model as a checkpoint for this trial
            trial_model_path = f"trial_{trial.number}_model.pth"
            torch.save(model.state_dict(), trial_model_path)
        else:
            patience += 1
            if patience >= max_patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    return best_val_acc

# Main function
def main():
    # Perform label encoding first
    num_classes = perform_label_encoding()
    print(f"Number of classes: {num_classes}")

    # Create the optuna study with A100-optimized settings
    study = optuna.create_study(direction='maximize', sampler=TPESampler())

    # Run fewer trials but with more GPU utilization
    n_trials = 30 # 10
    print(f"Running {n_trials} trials with larger batch sizes to maximize GPU usage")

    # Pass num_classes to objective function using a lambda function
    study.optimize(lambda trial: objective(trial, num_classes), n_trials=n_trials)

    # Get the best parameters
    best_params = study.best_params
    best_value = study.best_value
    print(f"Best trial: {study.best_trial.number}")
    print(f"Best validation accuracy: {best_value:.4f}")
    print(f"Best hyperparameters: {best_params}")

    # Load the best model from the best trial
    best_model = create_model(num_classes, best_params['dropout_rate'])
    best_model.load_state_dict(torch.load(f"trial_{study.best_trial.number}_model.pth"))

    # Save the best model
    torch.save(best_model.state_dict(), BEST_MODEL_PATH)
    print(f"Best model saved to {BEST_MODEL_PATH}")

    # Clean up trial model files
    for trial in study.trials:
        trial_model_path = f"trial_{trial.number}_model.pth"
        if os.path.exists(trial_model_path):
            os.remove(trial_model_path)

if __name__ == "__main__":
    main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[I 2025-03-19 18:27:09,007] A new study created in memory with name: no-name-f77ce727-e42c-448d-a347-3ceff3ebe626


Label encoding saved to /content/gdrive/My Drive/Projects/Multimodal/Datasets/label_to_idx-100.json
Number of classes: 4
Running 30 trials with larger batch sizes to maximize GPU usage


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 112MB/s]
<ipython-input-1-692ffc916b81>:313: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()
Training:   0%|          | 0/32 [00:00<?, ?it/s]<ipython-input-1-692ffc916b81>:230: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
Training: 100%|██████████| 32/32 [00:23<00:00,  1.34it/s]
<ipython-input-1-692ffc916b81>:256: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), amp.autocast():
Validating: 100%|██████████| 8/8 [00:05<00:00,  1.35it/s]


Epoch 1, Train Loss: 1.3531, Train Acc: 0.3633, Val Loss: 1.2100, Val Acc: 0.4844
GPU Memory: 0.11GB / 42.47GB


...

Epoch 3, Train Loss: 1.1645, Train Acc: 0.5391, Val Loss: 1.0463, Val Acc: 0.6406
GPU Memory: 0.40GB / 42.47GB


<ipython-input-1-692ffc916b81>:313: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()
Training:   0%|          | 0/8 [00:00<?, ?it/s]<ipython-input-1-692ffc916b81>:230: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
Training: 100%|██████████| 8/8 [00:23<00:00,  2.93s/it]
<ipython-input-1-692ffc916b81>:256: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), amp.autocast():
Validating: 100%|██████████| 2/2 [00:22<00:00, 11.44s/it]
[I 2025-03-19 19:39:33,294] Trial 28 pruned. 


Epoch 1, Train Loss: 1.5168, Train Acc: 0.2363, Val Loss: 1.4195, Val Acc: 0.1641
GPU Memory: 0.40GB / 42.47GB


<ipython-input-1-692ffc916b81>:313: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()
Training:   0%|          | 0/16 [00:00<?, ?it/s]<ipython-input-1-692ffc916b81>:230: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
Training: 100%|██████████| 16/16 [00:23<00:00,  1.47s/it]
<ipython-input-1-692ffc916b81>:256: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), amp.autocast():
Validating: 100%|██████████| 4/4 [00:11<00:00,  2.90s/it]
[I 2025-03-19 19:40:09,012] Trial 29 pruned. 


Epoch 1, Train Loss: 1.4562, Train Acc: 0.2363, Val Loss: 1.3876, Val Acc: 0.2734
GPU Memory: 0.40GB / 42.47GB
Best trial: 22
Best validation accuracy: 0.9375
Best hyperparameters: {'batch_size': 16, 'learning_rate': 0.0009712173013334415, 'dropout_rate': 0.2528655775853841, 'weight_decay': 5.127816354780079e-05}
Best model saved to /content/gdrive/My Drive/Projects/Multimodal/Datasets/best_resnet50_model-100.pth


## Predictions Phase

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import requests
from io import BytesIO
import json
import logging
import os
import time
from google.colab import drive
drive.mount('/content/gdrive')

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('prediction.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

class TestImageDataset(Dataset):
    """Custom Dataset for loading test images"""
    def __init__(self, csv_file, feature_col, transform=None):
        self.data = pd.read_csv(csv_file)
        self.feature_col = feature_col
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        try:
            img_url = self.data.iloc[idx][self.feature_col]

            # Download and open image
            response = requests.get(img_url, timeout=10)
            if response.status_code != 200:
                raise ValueError(f"Failed to fetch image: HTTP {response.status_code}")

            img = Image.open(BytesIO(response.content)).convert('RGB')

            if self.transform:
                img = self.transform(img)

            return img, idx

        except Exception as e:
            logger.error(f"Error loading image at index {idx}: {str(e)}")
            raise

def load_model_and_labels(model_path, label_to_idx_path):
    """Load the trained model and label mapping"""
    try:
        # Load label mapping
        with open(label_to_idx_path, 'r') as f:
            label_to_idx = json.load(f)

        # Create inverse mapping
        idx_to_label = {v: k for k, v in label_to_idx.items()}

        # Initialize model
        model = models.resnet50(weights=None)
        model.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(model.fc.in_features, len(label_to_idx))
        )

        # Load trained weights
        model.load_state_dict(torch.load(model_path))

        return model, idx_to_label

    except Exception as e:
        logger.error(f"Error loading model and labels: {str(e)}")
        raise

def predict_images(test_set_path, model_path, label_to_idx_path, batch_size,
                  prediction_col_name, output_path, feature_col='Image'):
    """
    Make predictions on test images and save results

    Parameters:
    - test_set_path: path to test CSV file
    - model_path: path to trained model weights
    - label_to_idx_path: path to label mapping JSON
    - batch_size: batch size for predictions
    - prediction_col_name: name for the new predictions column
    - output_path: path to save predictions CSV
    - feature_col: name of column containing image URLs

    Returns:
    - result_df: DataFrame with predictions
    - execution_time: Time taken for predictions in seconds
    - prediction_cost: Cost of predictions based on execution time
    """
    try:
        # Start timing
        start_time = time.time()

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        logger.info(f"Using device: {device}")

        # Load test data
        test_df = pd.read_csv(test_set_path)
        logger.info(f"Loaded test set with {len(test_df)} images")

        # Create transforms for test images
        test_transform = transforms.Compose([
            transforms.Resize((224, 224)),  # Standard ResNet input size
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225])
        ])

        # Create dataset and dataloader
        test_dataset = TestImageDataset(test_set_path, feature_col, test_transform)
        test_loader = DataLoader(test_dataset,
                               batch_size=batch_size,
                               shuffle=False,
                               num_workers=4)

        # Load model and label mapping
        model, idx_to_label = load_model_and_labels(model_path, label_to_idx_path)
        model = model.to(device)
        model.eval()

        # Make predictions
        predictions = []
        with torch.no_grad():
            for batch_images, batch_indices in test_loader:
                batch_images = batch_images.to(device)
                outputs = model(batch_images)
                _, predicted = torch.max(outputs.data, 1)

                # Convert indices to labels
                batch_predictions = [idx_to_label[idx.item()]
                                  for idx in predicted]

                # Store predictions with their indices
                for idx, pred in zip(batch_indices, batch_predictions):
                    predictions.append((idx.item(), pred))

        # Sort predictions by index to maintain original order
        predictions.sort(key=lambda x: x[0])
        predicted_labels = [pred[1] for pred in predictions]

        # Add predictions to dataframe
        test_df[prediction_col_name] = predicted_labels

        # Save results
        test_df.to_csv(output_path, index=False)

        # Calculate execution time and cost
        execution_time = time.time() - start_time
        prediction_cost = 0.000281392488 * execution_time

        logger.info(f"Predictions saved to {output_path}")
        logger.info(f"Prediction time: {execution_time:.2f} seconds")
        logger.info(f"Prediction cost: ${prediction_cost:.6f}")

        return test_df, execution_time, prediction_cost

    except Exception as e:
        logger.error(f"Error in prediction pipeline: {str(e)}")
        raise

absolute_path = "/content/gdrive/My Drive/Projects/Multimodal/"

if __name__ == "__main__":
    test_params = {
        'test_set_path': absolute_path + 'Datasets/Apple_test_set_100.csv',
        'model_path': absolute_path + 'Datasets/best_resnet50_model-100.pth',
        'label_to_idx_path': absolute_path + 'Datasets/label_to_idx-100.json',
        'batch_size': 16,
        'prediction_col_name': 'ResNet50-Predictions-Bayesian-Optimization',
        'output_path': absolute_path + 'Datasets/Apple-test_set_100_with_predictions.csv'
    }

    # Run predictions
    result_df, execution_time, prediction_cost = predict_images(**test_params)

    print("\nPrediction Results Summary:")
    print(f"Total prediction time: {execution_time:.2f} seconds")
    print(f"Total prediction cost: ${prediction_cost:.6f}")

Mounted at /content/gdrive

Prediction Results Summary:
Total prediction time: 23.64 seconds
Total prediction cost: $0.006653
